# EDA Analysis of Raw NBA DATA

In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 500)

# For Visualization
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'../data')


## Data
 - games_details.csv .. (each-game player stats for everyone on the roster)
 - games.csv .......... (each-game team stats: final scores, points scored, field-goal & free-throw percentages, etc...)
 - players.csv ........ (index of players' names and teams)
 - ranking.csv ........ (incremental daily record of standings, games played, won, lost, win%, home record, road record)
 - teams.csv .......... (index of team info such as city and arena names and also head coach) 

## games.csv

In [ ]:
TARGET = 'HOME_TEAM_WINS'
games = pd.read_csv(DATAPATH / "interim" / "games.csv")
games.head()

In [ ]:
games.info()

###
### Check for duplicates and nulls

In [ ]:
# duplicates
games.duplicated().sum()

In [ ]:
# null values
games.isnull().sum()

In [ ]:
#show the nulls
games[games.isna().any(axis=1)]

NOTE - 99 games from the early part of 2003 season missing data. 

NOTE - GAME_ID format: 

- 1st digit: 1=pre-season, 2=regular season, >2 = post season,  
- 2nd & 3rd digit: last two digits of season year (eg 103######## represents preseason game in 2003)

###
### Check distribution of missing games
Make sure these are fairly evenly distributed in case we decide to just delete them.

In [ ]:
# check distribution of missing games 
df_home = games[games.isna().any(axis=1)].groupby(['HOME_TEAM_ID'])['HOME_TEAM_ID'].count()
ax = df_home.plot.bar(x='HOME_TEAM_ID', y='count', rot=90, ylabel="No of Games Missing Data")

In [ ]:
# check distribution of missing games 
df_visitor = games[games.isna().any(axis=1)].groupby(['VISITOR_TEAM_ID'])['VISITOR_TEAM_ID'].count()
ax = df_visitor.plot.bar(x='VISITOR_TEAM_ID', y='count', rot=90, ylabel="No of Games Missing Data")

In [ ]:
df = pd.merge(df_home, df_visitor, right_index = True, left_index = True)
df.plot.bar(stacked = True, ylabel="No of Games Missing Data")

###
### Verify okay to drop certain fields
 - Confirm that GAMES_STATUS_TEXT always = "Final"
 - Confirm that HOME_TEAM_ID always = TEAM_ID-home
 - Confirm that VISITOR_TEAM_ID always = TEAM_ID-away

In [ ]:
#count all games not Final

games[games['GAME_STATUS_TEXT'] != 'Final'].shape[0]

###
### Describe the data


In [ ]:
games['GAME_DATE_EST'].str.split(" ").str[0]

In [ ]:
# fix data types first

games['GAME_DATE_EST'] = pd.to_datetime(games['GAME_DATE_EST'].str.split(" ").str[0])

category_fields = ['HOME_TEAM_ID', 'VISITOR_TEAM_ID', 'SEASON']

for cat in category_fields:
    games[cat] = games[cat].astype('category')

In [ ]:
games.describe(include = 'category').T

30 NBA teams and 19 seasons 

In [ ]:
games.describe().T.applymap('{:,.4f}'.format)

###
### Check anomalies
Some games seem to have a lot of points scored

In [ ]:
# max points = 168?
games[(games['PTS_home'] > 160) | (games['PTS_away'] > 160) ]

NOTE - Google search reveals that the 168 point games are overtime games

###
### Check Distributions


In [ ]:
value_list = ['PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FG_PCT_away', 
              'FG3_PCT_home', 'FG3_PCT_away',
              'FT_PCT_home', 'FT_PCT_away', 
              'REB_home', 'REB_away',
              'AST_home', 'AST_away', 
              'HOME_TEAM_WINS',
             ]

sns.set(rc={'figure.figsize':(25,25)})
for i, column in enumerate(value_list, 1):
    plt.subplot(4,4,i)
    p=sns.histplot(x=column, data=games, stat='count', kde=True)

###
### Group data to check season-by-season variation


In [ ]:
# summarize data for each season to check season by season variation

value_list = ['GAME_ID', 'HOME_TEAM_WINS', 'PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
              'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 
             ]
agg_dict = {'GAME_ID': len, 'HOME_TEAM_WINS': np.sum, 'PTS_home': np.mean, 'PTS_away': np.mean,
           'FG_PCT_home': np.mean, 'FT_PCT_home': np.mean, 'FG3_PCT_home': np.mean, 'AST_home': np.mean, 'REB_home': np.mean,
           'FG_PCT_away': np.mean, 'FT_PCT_away': np.mean, 'FG3_PCT_away': np.mean, 'AST_away': np.mean, 'REB_away': np.mean,
           }

df = pd.pivot_table(games,index=['SEASON'],
                    values=value_list,
                    aggfunc=agg_dict,
                   )
df = df.reset_index()
df = df.rename(columns={'GAME_ID': 'GAMES_PLAYED',})

df['HOME_WIN_RATIO'] = df['HOME_TEAM_WINS'] / df['GAMES_PLAYED']

# move more interesting columns to front
column1 = df.pop('GAMES_PLAYED')
column2 = df.pop('HOME_TEAM_WINS')
column3 = df.pop('HOME_WIN_RATIO')
df.insert(1,'GAMES_PLAYED', column1)
df.insert(2,'HOME_TEAM_WINS', column2)
df.insert(3,'HOME_WIN_RATIO', column3)

df

In [ ]:
value_list = ['PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FG_PCT_away', 
              'FG3_PCT_home', 'FG3_PCT_away',
              'FT_PCT_home', 'FT_PCT_away', 
              'REB_home', 'REB_away',
              'AST_home', 'AST_away', 
              'HOME_WIN_RATIO',
             ]
for i, column in enumerate(value_list, 1):
    plt.subplot(4,4,i)
    plt.xticks(rotation=65, horizontalalignment='right',)
    ax = sns.barplot(data=df, x='SEASON', y=column, color='blue')
 

NOTE - PTS, REB, AST have trended up the last several seasons, but Home win ratio is down

In [ ]:
df.describe().T

In [ ]:
value_list = ['PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FG_PCT_away', 
              'FG3_PCT_home', 'FG3_PCT_away',
              'FT_PCT_home', 'FT_PCT_away', 
              'REB_home', 'REB_away',
              'AST_home', 'AST_away', 
              'HOME_WIN_RATIO',
             ]

sns.set(rc={'figure.figsize':(25,25)})
for i, column in enumerate(value_list, 1):
    plt.subplot(4,4,i)
    p=sns.histplot(x=column, data=df, stat='count',bins = 20)

### 
### Group to check team-by-team variation for one season

In [ ]:
# summarize data for each team during 2021 to check team-by-team variation

games2021 = games[games['SEASON']==2021]

value_list = ['GAME_ID', 'HOME_TEAM_WINS', 'PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FT_PCT_home', 'FG3_PCT_home', 'AST_home', 'REB_home',
              'FG_PCT_away', 'FT_PCT_away', 'FG3_PCT_away', 'AST_away', 'REB_away', 
             ]
agg_dict = {'GAME_ID': len, 'HOME_TEAM_WINS': np.sum, 'PTS_home': np.mean, 'PTS_away': np.mean,
           'FG_PCT_home': np.mean, 'FT_PCT_home': np.mean, 'FG3_PCT_home': np.mean, 'AST_home': np.mean, 'REB_home': np.mean,
           'FG_PCT_away': np.mean, 'FT_PCT_away': np.mean, 'FG3_PCT_away': np.mean, 'AST_away': np.mean, 'REB_away': np.mean,
           }

df = pd.pivot_table(games2021,index=['HOME_TEAM_ID'],
                    values=value_list,
                    aggfunc=agg_dict,
                   )

df = df.reset_index()
df = df.rename(columns={'GAME_ID': 'GAMES_PLAYED',})

df['HOME_WIN_RATIO'] = df['HOME_TEAM_WINS'] / df['GAMES_PLAYED']

# move more interesting columns to front
column1 = df.pop('GAMES_PLAYED')
column2 = df.pop('HOME_TEAM_WINS')
column3 = df.pop('HOME_WIN_RATIO')
df.insert(1,'GAMES_PLAYED', column1)
df.insert(2,'HOME_TEAM_WINS', column2)
df.insert(3,'HOME_WIN_RATIO', column3)

df

In [ ]:
value_list = ['PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FG_PCT_away', 
              'FG3_PCT_home', 'FG3_PCT_away',
              'FT_PCT_home', 'FT_PCT_away', 
              'REB_home', 'REB_away',
              'AST_home', 'AST_away', 
              'HOME_WIN_RATIO',
             ]
for i, column in enumerate(value_list, 1):
    plt.subplot(4,4,i)
    plt.xticks(rotation=45, horizontalalignment='right',)
    ax = sns.barplot(data=df, x='HOME_TEAM_ID', y=column, color='blue')

In [ ]:
df.describe().T

In [ ]:
value_list = ['PTS_home', 'PTS_away', 
              'FG_PCT_home', 'FG_PCT_away', 
              'FG3_PCT_home', 'FG3_PCT_away',
              'FT_PCT_home', 'FT_PCT_away', 
              'REB_home', 'REB_away',
              'AST_home', 'AST_away', 
              'HOME_WIN_RATIO',
             ]

sns.set(rc={'figure.figsize':(25,25)})
for i, column in enumerate(value_list, 1):
    plt.subplot(4,4,i)
    p=sns.histplot(x=column, data=df, stat='count', bins = 20)

###
### Correlations

In [ ]:
# correlation bar chart

from matplotlib.colors import TwoSlopeNorm

df1 = games.drop(columns=['GAME_ID'])
#df1[cat_cols] = OrdinalEncoder().fit_transform(df1[cat_cols])

useful_columns =  df1.select_dtypes(include=['number']).columns

def get_redundant_pairs(df):
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0,df.shape[1]):
        for j in range(0,i+1):
            pairs_to_drop.add((cols[i],cols[j]))
    return pairs_to_drop

def get_correlations(df,n=30):
    au_corr = df.corr(method = 'spearman').unstack() #spearman used because not all data is normalized
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels = labels_to_drop).sort_values(ascending=False)
    top_n = au_corr[0:n]    
    bottom_n =  au_corr[-n:]
    top_corr = pd.concat([top_n, bottom_n])
    return top_corr

corrplot = get_correlations(df1[useful_columns])


fig, ax = plt.subplots(figsize=(15,10))
norm = TwoSlopeNorm(vmin=-1, vcenter=0, vmax =1)
colors = [plt.cm.RdYlGn(norm(c)) for c in corrplot.values]

print(corrplot)

corrplot.plot.barh(color=colors)

NOTE 
- FG_PCT and AST strongly correlated to PTS. 
- Opposing REB_PCT strongly negatively correlated to FG_PCT.

###
# ranking.csv

This dataset contains an incremental daily record of the conference standings of each team as it changes each day of the season, updating with current stats such as: 
- games played
- games won
- games lost
- winning percentage
- home record
- road record


In [ ]:
ranking = pd.read_csv(DATAPATH / "raw" / "ranking.csv")
ranking.head()

In [ ]:
ranking.info()

###
### Check for duplicates and nulls

In [ ]:
# duplicates
ranking.duplicated().sum()

In [ ]:
#null values
ranking.isnull().sum()

In [ ]:
# most rows have nulls in RETURNTOPLAY
# look at those that do not
ranking[ranking['RETURNTOPLAY'].notnull()]

In [ ]:
ranking[ranking['RETURNTOPLAY'] == 1]

In [ ]:
ranking[ranking['RETURNTOPLAY'] == 0]

In [ ]:
# are any in west conference?
df = ranking[ranking['RETURNTOPLAY'].notnull()]
df[df['CONFERENCE'] == 'West'].shape[0]

NOTE - Not sure what RETURNTOPLAY means, but it is limited to East Conference stats between 2020-03-12 and 2020-12-10, and is probably of almost no utility at the moment

In [ ]:
# any LEAGUE_ID not 0?
ranking[ranking['LEAGUE_ID'] != 0].shape[0]

NOTE - LEAGUE_ID always 0, can drop

In [ ]:
#these fields not needed for processing, analysis, or modeling

drop_fields = ['LEAGUE_ID', 'RETURNTOPLAY', 'TEAM']

#ranking = ranking.drop(drop_fields,axis=1)

###
### Describe the data


In [ ]:
# fix data types first

ranking['STANDINGSDATE'] = ranking['STANDINGSDATE'].astype('datetime64[ns]')

category_fields = ['TEAM_ID', 'CONFERENCE', 'SEASON_ID']

for cat in category_fields:
    ranking[cat] = ranking[cat].astype('category')

In [ ]:
ranking.describe(include = 'category').T

In [ ]:
# why 39 SEASON_IDs?
ranking['SEASON_ID'].value_counts()

In [ ]:
ranking[(ranking['SEASON_ID']==12021) & (ranking['G'] > 0)]

In [ ]:
df = ranking[(ranking['SEASON_ID']==12021)]
df.describe().T

NOTE - SEASON_ID beginning with 1 appears to be preseason games

In [ ]:
#flag preseaon stats
ranking['PRESEASON'] = ((ranking['SEASON_ID'].astype('int') < 20000).astype('int')).astype('category')
ranking

In [ ]:
#home record and road record need to be converted to numeric

ranking['HOME_W'] = ranking['HOME_RECORD'].apply(lambda x: x.split('-')[0]).astype('int')
ranking['HOME_L'] = ranking['HOME_RECORD'].apply(lambda x: x.split('-')[1]).astype('int')
ranking['HOME_W_PCT'] = ranking['HOME_W'] / ( ranking['HOME_W'] + ranking['HOME_L'] )

ranking['ROAD_W'] = ranking['ROAD_RECORD'].apply(lambda x: x.split('-')[0]).astype('int')
ranking['ROAD_L'] = ranking['ROAD_RECORD'].apply(lambda x: x.split('-')[1]).astype('int')
ranking['ROAD_W_PCT'] = ranking['ROAD_W'] / ( ranking['ROAD_W'] + ranking['ROAD_L'] )


In [ ]:
df = ranking[ranking['PRESEASON']==0]
df.describe().T

## Validation

In [ ]:
# check that key fields are always increasing since they represent running totals

df = ranking[ranking['PRESEASON']==0]

seasons = df["SEASON_ID"].unique().to_list()
teams = df["TEAM_ID"].unique().to_list()
fields = ['G','W','L','HOME_W','HOME_L','ROAD_W','ROAD_L'] 

for season in seasons:
    for team in teams:
        df = ranking[(ranking['SEASON_ID'] == season) & (ranking['TEAM_ID'] == team)]
        df = df.set_index('STANDINGSDATE')
        df = df.sort_index(ascending=True)
        
        for field in fields:
            increasing = df[field].is_monotonic_increasing
            if not(increasing):
                print("Not increasing:", season, team, field)



In [ ]:
#sample 5 records to validate against historical record on google

df = ranking[ranking['PRESEASON']==0]
df.sample(n=5)

In [ ]:
# make sure each date only has data for 30 teams
# from the later merge process in data processing, there appears to be an issue
df = ranking.groupby(['STANDINGSDATE']).count()
df = df.reset_index()
df
df[df['TEAM_ID'] > 30]

In [ ]:
df = ranking[ranking['STANDINGSDATE'] == '2020-12-28']
df['TEAM_ID'].value_counts()

In [ ]:
df[df['TEAM_ID'] == 1610612752]

In [ ]:
df = df[df['TEAM_ID'] == 1610612752]
df.duplicated()

###
## game_details.csv

In [ ]:
details = pd.read_csv(DATAPATH / "raw" / "games_details.csv")
details

In [ ]:
details.info()

###
### Check for duplicates and nulls

In [ ]:
# duplicates
details.duplicated().sum()

In [ ]:
#null values
details.isnull().sum()

In [ ]:
#the 105603 nulls across all the key stats seems most important
details[details['MIN'].isna()]

--    
**COMMENT field**

In [ ]:
# the COMMENT field seems to indicate when the player did not play
pd.set_option('display.max_rows', 20)
df = details[details['MIN'].isna()]
df['COMMENT'].value_counts()

In [ ]:
# DNP - Did not play, DND - did not dress, NWT - not with team
# are any of these null?
df['COMMENT'].isna().sum()

In [ ]:
#which one?
df[df['COMMENT'].isna()]

NOTE - records where all stats are missing indicates games where that player did not play. This is indicated in the COMMENT field except for the one record above

In [ ]:
#check pattern
df[df['COMMENT'].str[4:5] != '-']

NOTE - 1121 records do not follow the "XXX -" pattern

In [ ]:
#list the comments that do not follow the pattern
df['COMMENT'][df['COMMENT'].str[4:5] != '-'].value_counts()

In [ ]:
#list any comments when stats not null
df = details[details['MIN'].notna()]
df['COMMENT'].value_counts().sum()

NOTE - COMMENT only used when player did not play

--      
**MIN field**

Minutes played seems to have mixed data: integer minutes and MIN:SEC

In [ ]:
details['MIN'].value_counts()

In [ ]:
# see if any values are left if we filter out nulls, isdigit(), and strings with ":"
df = details[details['MIN'].notna()]
df = df[~df['MIN'].str.isdigit()]
df = df[~df['MIN'].str.contains(':')]
df
df[df['GAME_ID'] == 10800045]

NOTE - 12 records have negative minutes played

In [ ]:
# convert MIN:SEC to float

df = details.loc[details['MIN'].str.contains(':',na=False)]
df['MIN_whole'] = df['MIN'].apply(lambda x: x.split(':')[0].split('.')[0]).astype("int8")
df['MIN_seconds'] = df['MIN'].apply(lambda x: x.split(':')[1]).astype("int8")
df['MIN'] = df['MIN_whole'] + (df['MIN_seconds'] / 60)

details['MIN'].loc[details['MIN'].str.contains(':',na=False)] = df['MIN']
details['MIN'] = details['MIN'].astype("float64")

# convert negatives to positive
details['MIN'].loc[details['MIN'] < 0] = -(df['MIN'])

details


--     
**START_POSITION field**

In [ ]:
details['START_POSITION'].value_counts()

NOTE - maybe convert nulls to a character if player did play but did not start

### describe data

In [ ]:
details.describe(include = 'float').T.applymap('{:,.4f}'.format)

###
### Check distributions

In [ ]:
value_list = details.select_dtypes(include=['float']).columns.to_list()

df = details[details['MIN'].notna()]

sns.set(rc={'figure.figsize':(25,25)})
for i, column in enumerate(value_list, 1):
    plt.subplot(5,4,i)
    ax=sns.histplot(x=column, data=df, stat='count')

### check outliers 

**96 minutes in one game**

In [ ]:
#list games where players played more than 60 minutes
df.loc[df['MIN'] > 60]

**field goal attempts**

In [ ]:
df.loc[df['FGA'] > 40]

NOTE - according to a google search, the 50 FGA by Kobe Bryant is correct

**free throw attempts**

In [ ]:
df.loc[df['FTA'] > 30]

NOTE - according to a google search, the 39 FTA by Dwight Howard are correct